In [1]:
# Empezamos instalando algunas dependencias en google collab (podemos usar requirements.txt tmb):
!pip install crewai langchain huggingface_hub duckduckgo-search chromadb pandas langchain-community litellm


In [ ]:
# Autentificación con Hugging Face:
from huggingface_hub import login
login("")


In [3]:

from crewai import Agent
from crewai.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchRun

#  Definición del Tool:


class DuckSearchTool(BaseTool):
    name: str = "duckduckgo_search"
    description: str = "Realiza búsquedas web usando DuckDuckGo."

    def _run(self, query: str):
        search = DuckDuckGoSearchRun()
        return search.run(query)


#Definimos los Agentes:

search_tool = DuckSearchTool()

researcher = Agent(
    name="Researcher",
    role="AI Research Specialist",
    goal="Recolectar información actual y relevante desde la web.",
    backstory="Experto en análisis de fuentes públicas y rastreo de información técnica.",
    tools=[search_tool],
    llm="HuggingFaceH4/zephyr-7b-beta"
)

writer = Agent(
    name="Writer",
    role="Technical Writer",
    goal="Redactar un resumen estructurado de alrededor de 500 palabras.",
    backstory="Especialista en comunicación científica y síntesis de información compleja.",
    llm="HuggingFaceH4/zephyr-7b-beta"
)

reviewer = Agent(
    name="Reviewer",
    role="Research Quality Analyst",
    goal="Revisar precisión, coherencia y solidez del contenido final.",
    backstory="Editor académico con experiencia en verificación de hechos.",
    llm="microsoft/deberta-v3-small"
)


In [4]:
!pip install -U ddgs

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
results = search.run("bias in LLMs analysis")


In [6]:
#Realizamos algunos imports:

from huggingface_hub import InferenceClient
from google.colab import files
import os
import textwrap


In [ ]:
#Configuración:

HF_TOKEN = os.environ.get("")
HF_MODEL = "HuggingFaceH4/zephyr-7b-beta"   # Cambia si quieres otro modelo disponible en HF
client = InferenceClient(model=HF_MODEL, token=HF_TOKEN)

# Prompt base para pedir 500 palabras en Markdown (estructura: Intro, Hallazgos, Retos, Conclusión):

base_prompt = f"""

Eres un analista académico. Tu tarea es redactar un informe breve (~500 palabras) en formato Markdown
sobre el siguiente tema, utilizando los fragmentos de información provistos.

{results}


Requisitos:
- Mantener un tono académico y neutral.
- Integrar los fragmentos de evidencia de manera fluida (sin copiar literalmente).
- Organización clara y coherente.
- Redactar todo el texto en español.
- Extensión aproximada: 500 palabras.
- Todo en markdown.

Tema: Sesgos en modelos de lenguaje (Bias in LLMs)

Estructura:
1. Introducción
2. Hallazgos principales
3. Desafios éticos y técnicos
4. Conclusión
"""
respuesta = client.chat_completion(
    model="HuggingFaceH4/zephyr-7b-beta",
    messages=[{"role": "user", "content": base_prompt}],
    max_tokens=700,
    temperature=0.3
)

respuesta_contenido = respuesta["choices"][0]["message"]["content"]



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:

# Reseña:
reseña_prompt = f"""
Revisa el siguiente texto para asegurar que no haya errores de coherencia, factualidad y en general, de calidad académica.

Text:
{respuesta_contenido}
"""

reseña = client.chat_completion(
    model="HuggingFaceH4/zephyr-7b-beta",
    messages=[{"role": "user", "content": reseña_prompt}],
    max_tokens=400,
    temperature=0.3
)

reseña_contenido = reseña["choices"][0]["message"]["content"]

# Guardamos archivo

with open("research_summary.md", "w", encoding="utf-8") as f:
    f.write(respuesta_contenido)



In [13]:
#Para descargar el archivo:

from google.colab import files
files.download("research_summary.md")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>